In [ ]:
import json
import csv
import numpy as np
import matplotlib.pyplot as plt
from mp_api.client import MPRester

# Replace this with your Materials Project API Key
API_KEY = "YOUR_API_KEY_HERE"

def fetch_binary_transition_metal_compounds():
    TM_3d = ["Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn"]
    TM_4d = ["Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd"]
    TM_5d = ["Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg"]
    transition_metals = TM_3d + TM_4d + TM_5d

    nonmetals = ["O", "B", "C", "Si", "Ge", "N", "P", "As", "Sb", "Bi", "S", "Se", "Te", "F", "Cl", "Br", "I"]

    unique_results = {}

    with MPRester(API_KEY) as mpr:
        for metal in transition_metals:
            for nonmetal in nonmetals:
                elements = sorted([metal, nonmetal])
                chemsys_str = "-".join(elements)
                print(f"Querying chemical system: {chemsys_str}")
                try:
                    docs = mpr.summary.search(
                        chemsys=[chemsys_str],
                        band_gap=(0, 10),
                        fields=["material_id", "formula_pretty", "structure", "band_gap",
                                "formation_energy_per_atom", "vbm", "cbm", "efermi"],
                    )
                except Exception as e:
                    print(f"Error querying {chemsys_str}: {e}")
                    continue

                for doc in docs:
                    formula = doc.formula_pretty
                    formation_energy = doc.formation_energy_per_atom

                    entry = {
                        "material_id": doc.material_id,
                        "formula": formula,
                        "band_gap": doc.band_gap,
                        "formation_energy_per_atom": formation_energy,
                        "structure": doc.structure.as_dict() if doc.structure else None,
                        "vbm": doc.vbm,
                        "cbm": doc.cbm,
                        "efermi": doc.efermi,
                    }
                    if formula in unique_results:
                        if formation_energy < unique_results[formula]["formation_energy_per_atom"]:
                            unique_results[formula] = entry
                    else:
                        unique_results[formula] = entry
    
    binary_tm_compounds = list(unique_results.values())

    output_filename = "binary_tm_compounds_0-10eV.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(binary_tm_compounds, f, ensure_ascii=False, indent=2)
    print(f"Collected {len(binary_tm_compounds)} binary transition metal compounds meeting criteria, saved to {output_filename}")

if __name__ == "__main__":
    fetch_binary_transition_metal_compounds()